In [1]:
# <=== Click that Play button to run 

# This is a script to play, and to help you play and win at, Wordle.
# https://www.nytimes.com/games/wordle/index.html
# https://en.wikipedia.org/wiki/Wordle

# A note regarding duplicate letters: (From Wikipedia):
# Multiple instances of the same letter in a guess, such as the "o"s
# in "robot", will be colored green or yellow only if the letter also appears
# multiple times in the answer; otherwise, excess repeating letters will be
# colored gray (and it's *not* sequential, could have gray 'o' before green 'o')

# Backlog:
# Or, Search for TODO below in the code

# There is a list of potential target words (the A list), which seems to exclude declensions.
#   Use a PoS tagger to boost words that are equal to their own stem? (And non-proper-nouns, etc)
# There is a list of allowed guesses (the B list), which can be strategically useful to excluding letters.
# Use a decision tree / random forest?
# https://en.wikipedia.org/wiki/Random_forest

# Scoring
# Once I've already confirmed an S (maybe at a single pos),
# the remaining scores for other potential S letters in the word should be updated
# Is an HMM relevant here?

# eg slate=>price=>whine=>[guide,oxide]

# At the last iteration, 'guide' isn't more likely than 'oxide', they're equal. One
# can only say that guide is more likeley than oxide based on the fact that it
# contains letters that are more frequent in the dictionary as a whole. But, by
# the time we've eliminated all the other words, the last two words are equally
# likely to be the answer. It's just hypothetical to score "guide" higher, but not real.

# Consider not requiring hard-mode always?
# But that changes a lot of the assumptions that each reply contains all the inforamtion about subsequent candidates ...

# Onto G Collab sheet ? but sync with GitHub?

# TODO Auto mode: (with stats)
# Make it into a benchmark mode, that loops over the whole dictionary, and compute the average num guesses.
# So, that you can then evaluate alternative strategies/scoring across the whole dictionary.
# cf. https://freshman.dev/wordle/#/leaderboard
# And https://www.reddit.com/r/wordle/comments/s88iq4/a_wordle_bot_leaderboard/
# And make it print a running average while it's running ?
# Compare to NYT WordleBot https://www.nytimes.com/2022/04/07/upshot/wordle-bot-introduction.html

# TODO be more efficient with processing of duplicates

# Checkout strategy suggestions published by others:
# https://slate.com/technology/2022/01/wordle-how-to-win-strategy-crossword-experts.html
#   Alt strategy: maximize information gain of guess by not always including required letters.
#   What choice of letters gets closest to a 50% split of eligible words (max elimination rate)


import random
import re
from operator import itemgetter


def score_letters(word):
    """For choosing candidate starting words, based on common letter frequencies"""
    global opts
    global letter_freq_d
    score = 0.0
    used = set()
    for pos, letter in enumerate(word):

        letter = letter.lower()

        # Also count per-pos frequency (higher prio)
        # Pos is 1-based counting, since [0] counts "any position"
        score += 0.5 - abs(0.5 - letter_freq_d[letter][pos+1])

        # TODO rather than completely skip duplicate scoring, we could just linear downweight duplicates.
        # Because words do have duplicate letters, and we might get extra info by guessing for duplicates too.
        if letter not in used:

            # Scale this down, since per-pos score weighs more
            score += 0.05 * (0.5 - abs(0.5 - letter_freq_d[letter][0]))

        used.add(letter)
    return score


class dictd(dict):
    """A dict that allows dotted attr access implicitly, eg `x=d.mykey`"""
    def __setattr__(self, attr, value):
        self[attr] = value

    def __getattr__(self, attr):
        return self.get(attr, None)

    def sliced(dict, *keys):
        """Get a slice of a dict: sliced(all_customers, *new_customer_ids)"""
        return { k:dict[k] for k in keys if k in dict }


opts = dictd({
    'length' : 5,
    'scoring': 4,
    'top': 15,
})

words_str = 'aback abase abate abbey abbot abhor abide abled abode abort about above abuse abyss acorn acrid actor acute adage adapt adept admin admit adobe adopt adore adorn adult affix afire afoot afoul after again agape agate agent agile aging aglow agony agree ahead aider aisle alarm album alert algae alibi alien align alike alive allay alley allot allow alloy aloft alone along aloof aloud alpha altar alter amass amaze amber amble amend amiss amity among ample amply amuse angel anger angle angry angst anime ankle annex annoy annul anode antic anvil aorta apart aphid aping apnea apple apply apron aptly arbor ardor arena argue arise armor aroma arose array arrow arson artsy ascot ashen aside askew assay asset atoll atone attic audio audit augur aunty avail avert avian avoid await awake award aware awash awful awoke axial axiom axion azure bacon badge badly bagel baggy baker baler balmy banal banjo barge baron basal basic basil basin basis baste batch bathe baton batty bawdy bayou beach beady beard beast beech beefy befit began begat beget begin begun being belch belie belle belly below bench beret berry berth beset betel bevel bezel bible bicep biddy bigot bilge billy binge bingo biome birch birth bison bitty black blade blame bland blank blare blast blaze bleak bleat bleed bleep blend bless blimp blind blink bliss blitz bloat block bloke blond blood bloom blown bluer bluff blunt blurb blurt blush board boast bobby boney bongo bonus booby boost booth booty booze boozy borax borne bosom bossy botch bough boule bound bowel boxer brace braid brain brake brand brash brass brave bravo brawl brawn bread break breed briar bribe brick bride brief brine bring brink briny brisk broad broil broke brood brook broom broth brown brunt brush brute buddy budge buggy bugle build built bulge bulky bully bunch bunny burly burnt burst bused bushy butch butte buxom buyer bylaw cabal cabby cabin cable cacao cache cacti caddy cadet cagey cairn camel cameo canal candy canny canoe canon caper caput carat cargo carol carry carve caste catch cater catty caulk cause cavil cease cedar cello chafe chaff chain chair chalk champ chant chaos chard charm chart chase chasm cheap cheat check cheek cheer chess chest chick chide chief child chili chill chime china chirp chock choir choke chord chore chose chuck chump chunk churn chute cider cigar cinch circa civic civil clack claim clamp clang clank clash clasp class clean clear cleat cleft clerk click cliff climb cling clink cloak clock clone close cloth cloud clout clove clown cluck clued clump clung coach coast cobra cocoa colon color comet comfy comic comma conch condo conic copse coral corer corny couch cough could count coupe court coven cover covet covey cower coyly crack craft cramp crane crank crash crass crate crave crawl craze crazy creak cream credo creed creek creep creme crepe crept cress crest crick cried crier crime crimp crisp croak crock crone crony crook cross croup crowd crown crude cruel crumb crump crush crust crypt cubic cumin curio curly curry curse curve curvy cutie cyber cycle cynic daddy daily dairy daisy dally dance dandy datum daunt dealt death debar debit debug debut decal decay decor decoy decry defer deign deity delay delta delve demon demur denim dense depot depth derby deter detox deuce devil diary dicey digit dilly dimly diner dingo dingy diode dirge dirty disco ditch ditto ditty diver dizzy dodge dodgy dogma doing dolly donor donut dopey doubt dough dowdy dowel downy dowry dozen draft drain drake drama drank drape drawl drawn dread dream dress dried drier drift drill drink drive droit droll drone drool droop dross drove drown druid drunk dryer dryly duchy dully dummy dumpy dunce dusky dusty dutch duvet dwarf dwell dwelt dying eager eagle early earth easel eaten eater ebony eclat edict edify eerie egret eight eject eking elate elbow elder elect elegy elfin elide elite elope elude email embed ember emcee empty enact endow enema enemy enjoy ennui ensue enter entry envoy epoch epoxy equal equip erase erect erode error erupt essay ester ether ethic ethos etude evade event every evict evoke exact exalt excel exert exile exist expel extol extra exult eying fable facet faint fairy faith false fancy fanny farce fatal fatty fault fauna favor feast fecal feign fella felon femme femur fence feral ferry fetal fetch fetid fetus fever fewer fiber ficus field fiend fiery fifth fifty fight filer filet filly filmy filth final finch finer first fishy fixer fizzy fjord flack flail flair flake flaky flame flank flare flash flask fleck fleet flesh flick flier fling flint flirt float flock flood floor flora floss flour flout flown fluff fluid fluke flume flung flunk flush flute flyer foamy focal focus foggy foist folio folly foray force forge forgo forte forth forty forum found foyer frail frame frank fraud freak freed freer fresh friar fried frill frisk fritz frock frond front frost froth frown froze fruit fudge fugue fully fungi funky funny furor furry fussy fuzzy gaffe gaily gamer gamma gamut gassy gaudy gauge gaunt gauze gavel gawky gayer gayly gazer gecko geeky geese genie genre ghost ghoul giant giddy gipsy girly girth given giver glade gland glare glass glaze gleam glean glide glint gloat globe gloom glory gloss glove glyph gnash gnome godly going golem golly gonad goner goody gooey goofy goose gorge gouge gourd grace grade graft grail grain grand grant grape graph grasp grass grate grave gravy graze great greed green greet grief grill grime grimy grind gripe groan groin groom grope gross group grout grove growl grown gruel gruff grunt guard guava guess guest guide guild guile guilt guise gulch gully gumbo gummy guppy gusto gusty gypsy habit hairy halve handy happy hardy harem harpy harry harsh haste hasty hatch hater haunt haute haven havoc hazel heady heard heart heath heave heavy hedge hefty heist helix hello hence heron hilly hinge hippo hippy hitch hoard hobby hoist holly homer honey honor horde horny horse hotel hotly hound house hovel hover howdy human humid humor humph humus hunch hunky hurry husky hussy hutch hydro hyena hymen hyper icily icing ideal idiom idiot idler idyll igloo iliac image imbue impel imply inane inbox incur index inept inert infer ingot inlay inlet inner input inter intro ionic irate irony islet issue itchy ivory jaunt jazzy jelly jerky jetty jewel jiffy joint joist joker jolly joust judge juice juicy jumbo jumpy junta junto juror kappa karma kayak kebab khaki kinky kiosk kitty knack knave knead kneed kneel knelt knife knock knoll known koala krill label labor laden ladle lager lance lanky lapel lapse large larva lasso latch later lathe latte laugh layer leach leafy leaky leant leapt learn lease leash least leave ledge leech leery lefty legal leggy lemon lemur leper level lever libel liege light liken lilac limbo limit linen liner lingo lipid lithe liver livid llama loamy loath lobby local locus lodge lofty logic login loopy loose lorry loser louse lousy lover lower lowly loyal lucid lucky lumen lumpy lunar lunch lunge lupus lurch lurid lusty lying lymph lyric macaw macho macro madam madly mafia magic magma maize major maker mambo mamma mammy manga mange mango mangy mania manic manly manor maple march marry marsh mason masse match matey mauve maxim maybe mayor mealy meant meaty mecca medal media medic melee melon mercy merge merit merry metal meter metro micro midge midst might milky mimic mince miner minim minor minty minus mirth miser missy mocha modal model modem mogul moist molar moldy money month moody moose moral moron morph mossy motel motif motor motto moult mound mount mourn mouse mouth mover movie mower mucky mucus muddy mulch mummy munch mural murky mushy music musky musty myrrh nadir naive nanny nasal nasty natal naval navel needy neigh nerdy nerve never newer newly nicer niche niece night ninja ninny ninth noble nobly noise noisy nomad noose north nosey notch novel nudge nurse nutty nylon nymph oaken obese occur ocean octal octet odder oddly offal offer often olden older olive ombre omega onion onset opera opine opium optic orbit order organ other otter ought ounce outdo outer outgo ovary ovate overt ovine ovoid owing owner oxide ozone paddy pagan paint paler palsy panel panic pansy papal paper parer parka parry parse party pasta paste pasty patch patio patsy patty pause payee payer peace peach pearl pecan pedal penal pence penne penny perch peril perky pesky pesto petal petty phase phone phony photo piano picky piece piety piggy pilot pinch piney pinky pinto piper pique pitch pithy pivot pixel pixie pizza place plaid plain plait plane plank plant plate plaza plead pleat plied plier pluck plumb plume plump plunk plush poesy point poise poker polar polka polyp pooch poppy porch poser posit posse pouch pound pouty power prank prawn preen press price prick pride pried prime primo print prior prism privy prize probe prone prong proof prose proud prove prowl proxy prude prune psalm pubic pudgy puffy pulpy pulse punch pupil puppy puree purer purge purse pushy putty pygmy quack quail quake qualm quark quart quash quasi queen queer quell query quest queue quick quiet quill quilt quirk quite quota quote quoth rabbi rabid racer radar radii radio rainy raise rajah rally ralph ramen ranch randy range rapid rarer raspy ratio ratty raven rayon razor reach react ready realm rearm rebar rebel rebus rebut recap recur recut reedy refer refit regal rehab reign relax relay relic remit renal renew repay repel reply rerun reset resin retch retro retry reuse revel revue rhino rhyme rider ridge rifle right rigid rigor rinse ripen riper risen riser risky rival river rivet roach roast robin robot rocky rodeo roger rogue roomy roost rotor rouge rough round rouse route rover rowdy rower royal ruddy ruder rugby ruler rumba rumor rupee rural rusty sadly safer saint salad sally salon salsa salty salve salvo sandy saner sappy sassy satin satyr sauce saucy sauna saute savor savoy savvy scald scale scalp scaly scamp scant scare scarf scary scene scent scion scoff scold scone scoop scope score scorn scour scout scowl scram scrap scree screw scrub scrum scuba sedan seedy segue seize semen sense sepia serif serum serve setup seven sever sewer shack shade shady shaft shake shaky shale shall shalt shame shank shape shard share shark sharp shave shawl shear sheen sheep sheer sheet sheik shelf shell shied shift shine shiny shire shirk shirt shoal shock shone shook shoot shore shorn short shout shove shown showy shrew shrub shrug shuck shunt shush shyly siege sieve sight sigma silky silly since sinew singe siren sissy sixth sixty skate skier skiff skill skimp skirt skulk skull skunk slack slain slang slant slash slate sleek sleep sleet slept slice slick slide slime slimy sling slink sloop slope slosh sloth slump slung slunk slurp slush slyly smack small smart smash smear smell smelt smile smirk smite smith smock smoke smoky smote snack snail snake snaky snare snarl sneak sneer snide sniff snipe snoop snore snort snout snowy snuck snuff soapy sober soggy solar solid solve sonar sonic sooth sooty sorry sound south sower space spade spank spare spark spasm spawn speak spear speck speed spell spelt spend spent sperm spice spicy spied spiel spike spiky spill spilt spine spiny spire spite splat split spoil spoke spoof spook spool spoon spore sport spout spray spree sprig spunk spurn spurt squad squat squib stack staff stage staid stain stair stake stale stalk stall stamp stand stank stare stark start stash state stave stead steak steal steam steed steel steep steer stein stern stick stiff still stilt sting stink stint stock stoic stoke stole stomp stone stony stood stool stoop store stork storm story stout stove strap straw stray strip strut stuck study stuff stump stung stunk stunt style suave sugar suing suite sulky sully sumac sunny super surer surge surly sushi swami swamp swarm swash swath swear sweat sweep sweet swell swept swift swill swine swing swirl swish swoon swoop sword swore sworn swung synod syrup tabby table taboo tacit tacky taffy taint taken taker tally talon tamer tango tangy taper tapir tardy tarot taste tasty tatty taunt tawny teach teary tease teddy teeth tempo tenet tenor tense tenth tepee tepid terra terse testy thank theft their theme there these theta thick thief thigh thing think third thong thorn those three threw throb throw thrum thumb thump thyme tiara tibia tidal tiger tight tilde timer timid tipsy titan tithe title toast today toddy token tonal tonga tonic tooth topaz topic torch torso torus total totem touch tough towel tower toxic toxin trace track tract trade trail train trait tramp trash trawl tread treat trend triad trial tribe trice trick tried tripe trite troll troop trope trout trove truce truck truer truly trump trunk truss trust truth tryst tubal tuber tulip tulle tumor tunic turbo tutor twang tweak tweed tweet twice twine twirl twist twixt tying udder ulcer ultra umbra uncle uncut under undid undue unfed unfit unify union unite unity unlit unmet unset untie until unwed unzip upper upset urban urine usage usher using usual usurp utile utter vague valet valid valor value valve vapid vapor vault vaunt vegan venom venue verge verse verso verve vicar video vigil vigor villa vinyl viola viper viral virus visit visor vista vital vivid vixen vocal vodka vogue voice voila vomit voter vouch vowel vying wacky wafer wager wagon waist waive waltz warty waste watch water waver waxen weary weave wedge weedy weigh weird welch welsh whack whale wharf wheat wheel whelp where which whiff while whine whiny whirl whisk white whole whoop whose widen wider widow width wield wight willy wimpy wince winch windy wiser wispy witch witty woken woman women woody wooer wooly woozy wordy world worry worse worst worth would wound woven wrack wrath wreak wreck wrest wring wrist write wrong wrote wrung wryly yacht yearn yeast yield young youth zebra zesty zonal'
words_left = words_str.split()
words_left_n = len(words_left)


################################################################################

# Collect indexes to find words matching certain criteria.
# This DS is a dict[list[set[]]]
# The dict tracks the letters
# The list tracks what position the letter is in (0 for wildcard/yellow position)
# The set tracks the words that meet that criteria.
# candidates['y'][2] contains 'word' # means the letter is the second (1-based) letter in the word

lookup = dict()

for word in words_left:
    # TODO split the rest of this out and just recompute for each round?
    # Then could also more easily just count occurrences of this letter over all letter-positions left
    for pos, letter in enumerate(word):
        lookup[letter] = lookup.get(letter) or [ set() for i in range(opts.length+1) ]

        # TODO
        # n_positions_with_letter[letter] =+ 1

        # Note, this word is a candidate for (green, positioned) letter (1-based)
        lookup[letter][pos+1].add(word)

# TODO
# Best way to track which letter frequencies?
# n_words_with_this_letter_at_this_pos / n_words
# n_words_with_this_letter_at_any_pos  / n_words # Union of the words with this letter
# n_letters_that_are_this_letter / n_letters # Over all words, eg (union - intersection) * opts.length

# TODO factor this out, and consider re-computing it upon each generation, based on the remaining words?
letter_freq_d = dict()
for letter in lookup:
    s = set()
    letter_freq_d[letter] = [ 0 for i in range(opts.length+1) ]
    # 1-based counting of letter positions in word
    for pos in range(1,opts.length+1):
        letter_freq_d[letter][pos] = len(lookup[letter][pos]) / words_left_n
        s = s | lookup[letter][pos]

    # What fraction of words have this letter in any pos
    letter_freq_d[letter][0] = len(s) / words_left_n


# This basically implements hard-mode by default, where confirmed letters are subsequently required.
# Keep track of multiples/duplicates required
min_letters = dict()
blacklist_words = set()
guesses_n = 0
guess = ''


while len(words_left) > 0:

    # Go over remaining candidates
    words_left = set()
    for letter in lookup:
        for pos in range(opts.length):
            words_left = words_left | lookup[letter][pos+1]
            # Really not efficient, but we need to ensure that wildcard letters present:
            for w in lookup[letter][pos+1]:
                for l in min_letters:
                    if w.count(l) < min_letters[l]:
                        blacklist_words.add(w)

    words_left = words_left - blacklist_words

    guesses_n += 1
    print()
    print(f"Round: {guesses_n}")
    print(f"Left:  {len(words_left)}")
    print()

    if not words_left:
        print('None')
        exit()
        continue

    scores = dict()
    for word in words_left:
        # Word freq. Note, this just makes the word more likely to be in the Wordle
        # dictionary, since there's some threshold for excluding less common words.
        # However, given two words in the dictionary, that one is twice as frequent
        # as the other doesn't mean it's more likely to be the target word.

        # from wordfreq import zipf_frequency
        # by_word = zipf_frequency(word, 'en')

        by_word = 0
        # TODO flag words that pass criteria for being target words (stem, no lead cap in EN)

        # Sum of freq of (unique) letters
        by_letter = score_letters(word)

        # A slightly more comparable scale:
        # by_combined = 10 * by_letter + by_word
        by_combined = by_letter + by_word

        scores[word] = { 'word': word, 'by_word': by_word, 'by_letter': by_letter, 'by_combined': by_combined, }

    # Sort for top N objs by key
    scores_sorted = sorted(scores.values(), key=itemgetter('by_combined'), reverse=True)
    if opts.top:
        # Print headings
        print(f"{'lett':>7} {'word':>7} {'combo':>7}")
    for s in scores_sorted[:opts.top]:
        print(f"{s['by_letter']:7.4f} {s['by_word']:7.1f} {s['by_combined']:7.4f} {s['word']:20s}")

    print()

    if len(words_left) == 1:
        exit()
        words_left.pop()
        continue

    guess = None
    if opts.auto:
        # Was an explicit starting word override given?
        if opts.start:
            scores_sorted.insert(0, { 'word': opts.start } )
            opts.start = None
        # Auto guess the top-scoring remaining word
        guess = scores_sorted[0]['word']
        print(f"Guess:  {guess}")

    while not guess:
        try:
            guess = input(f"Guess:  ")
        except:
            print()
            exit()

        match = re.match('\s*([+]?)\s*(.*)\s*', guess)
        if match:
            force, guess = match.groups()
        if guess not in words_left :
            # It's a bad guess, because its's already excluded, but allowed, so just warn
            # This is not a valid word in the original dictionary
            guess = None

    # Each letter in the reply has a corresponding operator code: exact (+), wild (*), miss (-)
    reply_ops = [ None for i in range(opts.length) ]

    # eg:  c+a*n-a-l*
    # Which means (0-based index of these chars):
    #   0-1 There's a 'c' at pos 1 (and maybe elsewehere?)
    #   2-3 There's an (or more) 'a' but not at pos 2 (pos 2 out of 5, 1-based)
    #   4-5 There is no 'n'
    #   6-7 There are no *additional* 'a' letters (i.e. just the previously found)
    #   8-9 There's an (or more) 'l', but not at pos 5

    # For display feedback only
    reply = ''

    if opts.target:
        # For tracking duplicate letters in target and guess
        remaining_letters = dict()
        for l in opts.target:
            remaining_letters[l] = remaining_letters.get(l) or 0
            remaining_letters[l] += 1

        # Just looking for exact matches in the first iteration.
        # This is because we need to prioritize scoring exact matches (+) before wilds (*).
        for pos, l in enumerate(guess):
            if l == opts.target[pos]:
                reply_ops[pos] = '+'
                remaining_letters[l] -= 1

        # Now see if there are any duplicate chars left in target for any wild matches (*)
        for pos, l in enumerate(guess):
            if reply_ops[pos]:
                # Already had an exact match
                continue
            elif l in remaining_letters and remaining_letters[l] > 0:
                reply_ops[pos] = '*'
                remaining_letters[l] -= 1
            else:
                reply_ops[pos] = '-'

        reply = ''.join(reply_ops)


    if reply:
        print("Reply: ", reply)
    else:
        while not reply:
            try:
                reply = input("Reply:  ")
            except:
                print()
                exit()

            if len(reply) != opts.length or not re.match('^[yon+*_-]+$', reply):
                reply = None
                beep()

    if (opts.target and guess == opts.target) or re.match('^[y+]+$', reply):
        print(f"\nFound: {guesses_n:2d} tries")
        exit()

    # Note if a letter was never seen (and no later positions), as then gray means not present at all
    letter_maybe_del = set()

    # TODO consider if it's more efficient to just maintain a regex and filter out from `remaining` in each iteration?
    # Two masks:
    # filter_in  = [ 's', '.', 'r', '.', '.' ]
    # filter_out = [
    #     [], [a,o,p], [f], [], [t,m]
    # ]
    # TODO 
    # The only missing info then is when we know the (min) count of duplicate letters, eg 2+ of 'e'
    # Guess:  semen
    # Reply:  +*_+_
    # Then there may be no remaining candidates that don't have 2+ 'e'

    # Count of required letters, based on previous replies.
    # This assumes hard-mode (that all previous info is used in each subsequent guess)
    min_letters = dict()

    for pos in range(opts.length):
        letter = guess[pos]
        op     = reply[pos]

        # Note, the below [pos+1] syntax is because 1-based counting in the target word
        if op in 'y+': # yes
            # Letter is present, at this position.
            min_letters[letter] = min_letters.get(letter) or 0
            min_letters[letter] += 1
            # But maybe also at other positions ... so, don't delete those yet.
            ...
            # However, no *other* letter is at *this* pos, so delete all of those.
            for l in lookup:
                if l != letter:
                    blacklist_words = blacklist_words | lookup[l][pos+1]
                    lookup[l][pos+1] = set()
        elif op in 'o*': # other
            # Letter is still a candidate, but not at this pos.
            # (Might still have (multiple) occurrences of this elsewhere. Don't remove those.)
            blacklist_words = blacklist_words | lookup[letter][pos+1]
            lookup[letter][pos+1] = set()
            # This letter is now required at some/any other pos
            min_letters[letter] = min_letters.get(letter) or 0
            min_letters[letter] += 1
        elif op in 'n-_': # no
            # (The additional '_' is just to also allow to keep the Shift key pressed for all op chars.)
            # Letter is not present in this position:
            blacklist_words = blacklist_words | lookup[letter][pos+1]
            lookup[letter][pos+1] = set()

            # Letter *maybe* not present at any other position, but maybe *later* in the word:
            # (Apparently the green letters take priority over the yellow/grey letters).
            letter_maybe_del.add(letter)
            # If letter has no (more) occurrences (including *later* occurrences):
            # Counter-example: target 'shake', but given 's+e-r-v-e+' (clearly not sequential)
            # Because the 'e' in pos 2 is grey, even before the (green) 'e' at pos 5 was processed.

    # After checking each pos for each letter, which were gray once, but otherwise not green later:
    for letter in letter_maybe_del:
        # Did we then find it later as a '*' or '+' letter?
        if letter not in min_letters:
            # Then blacklist all those words with this letter anywhere
            for s in lookup[letter]:
                blacklist_words = blacklist_words | s
            lookup[letter] = [ set() for i in range(opts.length+1)]




Round: 1
Left:  2309

   lett    word   combo
 0.7044     0.0  0.7044 slate               
 0.6843     0.0  0.6843 saute               
 0.6842     0.0  0.6842 sauce               
 0.6838     0.0  0.6838 shale               
 0.6827     0.0  0.6827 share               
 0.6814     0.0  0.6814 slice               
 0.6775     0.0  0.6775 crane               
 0.6708     0.0  0.6708 suite               
 0.6674     0.0  0.6674 shine               
 0.6666     0.0  0.6666 saint               
 0.6643     0.0  0.6643 saner               
 0.6617     0.0  0.6617 snare               
 0.6615     0.0  0.6615 crate               
 0.6611     0.0  0.6611 stale               
 0.6600     0.0  0.6600 stare               

Guess:  clerk
Reply:  yyyyy

Found:  1 tries

Round: 2
Left:  1

   lett    word   combo
 0.4318     0.0  0.4318 clerk               

